# Imports

In [48]:
import pandas as pd
import numpy as np
import os
from glob import glob
from typing import List

# Files

In [ ]:
src_path = os.path.join("/", "Volumes", "owen", "datas", "rplace")
src_glob = os.path.join(src_path, "2022_place_canvas_history-*.csv")
dest_path = os.path.join(src_path)

In [56]:
files = glob(src_glob)
files

['/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000000.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000001.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000052.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000002.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000003.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000004.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000005.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000006.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000007.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000008.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000009.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000010.csv',
 '/Volumes/owen/datas/rplace/2022_place_canvas_history-000000000013.csv',
 '/Volumes/owen/datas/rplace/2022_plac

# Data processing

In [50]:
def count_clicks(filename: str):
    df_place = pd.read_csv(filename)
    df_place.reset_index(inplace=True)
    df_pixel_click_count = df_place.groupby("coordinate").count()
    return df_pixel_click_count[["index"]].rename({"index": "count"}, axis=1)

In [51]:
def merge_count_clicks(files: List[str]):
    df: pd.DataFrame = count_clicks(files[0])

    for i in range(1, len(files)):
        print(f"Adding {files[i]} ({i / len(files) * 100}%)")
        df_to_add = count_clicks(files[i])
        df = df.add(df_to_add, fill_value=1)
    
    return df.sort_values("index", ascending=False)

In [52]:
df = merge_count_clicks(files[:])
df

Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000001.csv (1.2658227848101267%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000052.csv (2.5316455696202533%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000002.csv (3.79746835443038%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000003.csv (5.063291139240507%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000004.csv (6.329113924050633%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000005.csv (7.59493670886076%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000006.csv (8.860759493670885%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000007.csv (10.126582278481013%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000008.csv (11.39240506329114%)
Adding /Volumes/owen/datas/rplace/2022_place_canvas_history-000000000009.csv (12.658227848101266%)
Adding /Volumes/ow

,index,timestamp,user_id,pixel_color
coordinate,,,,
"0,0",98807.0,98807.0,98807.0,98807.0
"0,1",9592.0,9592.0,9592.0,9592.0
"0,10",323.0,323.0,323.0,323.0
"0,100",138.0,138.0,138.0,138.0
"0,1000",231.0,231.0,231.0,231.0
...,...,...,...,...
"999,995",337.0,337.0,337.0,337.0
"999,996",418.0,418.0,418.0,418.0
"999,997",922.0,922.0,922.0,922.0


In [1]:
df.to_csv(os.path.join(dest_path, "clicks_count.csv"))

NameError: name 'df' is not defined

In [53]:
def clicks_count_to_matrix(df: pd.DataFrame):
    m = np.zeros((2000, 2000))

    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            try:
                v = df.loc[f"{j},{i}"]
                if v["index"] != None:
                    m[i][j] = v["index"]
            except:
                pass

    return m

In [54]:
m = clicks_count_to_matrix(df)
m

array([[98807.,  9592.,  3888., ...,   738.,  1716., 22797.],
       [11294., 12083.,  1840., ...,   731.,  2052.,  2319.],
       [ 4721.,  2152.,  4456., ...,   857.,   665.,   862.],
       ...,
       [ 1464.,   733.,  1066., ...,   584.,   448.,  1352.],
       [ 3722.,  2184.,  1321., ...,   410.,  1701.,  3641.],
       [30895.,  3268.,  1624., ...,   776.,  2701., 31471.]])

In [59]:
np.save(os.path.join(dest_path, "clicks_count_matrix"), m)